# Chapter 7: Autoencoder for Gene Expression Analysis

Welcome to unsupervised learning with autoencoders! 🧬

In this notebook, we'll explore a different type of deep learning: **unsupervised learning**. We'll use autoencoders to analyze gene expression data and discover hidden patterns.

## 🎯 The Biological Problem

### What is Gene Expression Data?

Gene expression tells us how active each gene is in a cell:

- **Measurement:** RNA sequencing (RNA-seq) or microarrays
- **Data format:** For each sample (patient, cell, tissue), measure expression of ~20,000 genes
- **Result:** A matrix where each row is a sample, each column is a gene

**Example:**
```
Sample    Gene1  Gene2  Gene3  ... Gene20000
Patient1   5.2    0.1    8.9   ...    2.3
Patient2   5.5    0.2    8.7   ...    2.1
```

### The Challenge: Too Many Dimensions!

- **20,000 dimensions** (one per gene) is impossible to visualize or analyze directly
- Many genes are correlated (they work together)
- We want to find the **underlying patterns** that explain the data

### Traditional Approach: PCA

Principal Component Analysis (PCA) is the classical method:
- Linear dimensionality reduction
- Fast and interpretable
- But: Only finds linear relationships

### Deep Learning Approach: Autoencoders

Autoencoders are neural networks that:
- Learn to compress data (many dimensions → few dimensions)
- Then reconstruct the original data
- Can capture **non-linear** relationships

**Analogy:** Like learning to describe a movie (2 hour video) in a few sentences (compression), then using those sentences to recreate the plot (reconstruction).

## 📚 What You'll Learn

1. **Autoencoder Architecture:** Encoder → Bottleneck → Decoder
2. **Latent Space:** The compressed representation in the middle
3. **Dimensionality Reduction:** From 20,000 genes to 2-10 dimensions
4. **Clustering:** Finding groups of similar samples
5. **Comparison with PCA:** When to use each method

## 🔧 Applications in Biology

- **Disease subtyping:** Find different types of cancer based on gene expression
- **Biomarker discovery:** Identify genes that distinguish healthy vs diseased
- **Cell type identification:** Classify cells in single-cell RNA-seq data
- **Data visualization:** Plot high-dimensional data in 2D

Let's compress some genes! 🚀

---


## 1. Setup and Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from tqdm import tqdm

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Generate Synthetic Gene Expression Data

We'll simulate gene expression data for three different conditions:
- **Condition A**: Healthy samples
- **Condition B**: Disease state 1
- **Condition C**: Disease state 2

Each condition will have characteristic gene expression patterns.

In [ ]:
def generate_gene_expression_data(n_samples=300, n_genes=2000, n_informative=200):
    """
    Generate synthetic gene expression data.
    
    Args:
        n_samples: Number of samples per condition
        n_genes: Total number of genes
        n_informative: Number of genes that differ between conditions
    
    Returns:
        data: Gene expression matrix (samples x genes)
        labels: Sample labels (0, 1, 2 for three conditions)
        gene_names: List of gene names
    """
    
    # Generate gene names
    gene_names = [f"Gene_{i:04d}" for i in range(n_genes)]
    
    all_data = []
    all_labels = []
    
    for condition in range(3):
        # Base expression: log-normal distribution (typical for RNA-seq)
        base_expression = np.random.lognormal(mean=5, sigma=1.5, size=(n_samples, n_genes))
        
        # Add condition-specific patterns to informative genes
        informative_indices = np.random.choice(n_genes, n_informative, replace=False)
        
        if condition == 0:  # Healthy
            # Slightly elevated expression in some genes
            base_expression[:, informative_indices[:100]] *= np.random.uniform(1.5, 2.5, 100)
        
        elif condition == 1:  # Disease 1
            # Upregulated inflammatory genes
            base_expression[:, informative_indices[:50]] *= np.random.uniform(3, 5, 50)
            # Downregulated housekeeping genes
            base_expression[:, informative_indices[50:100]] *= np.random.uniform(0.2, 0.5, 50)
        
        else:  # Disease 2
            # Different pattern - metabolic changes
            base_expression[:, informative_indices[100:150]] *= np.random.uniform(4, 6, 50)
            base_expression[:, informative_indices[150:200]] *= np.random.uniform(0.1, 0.3, 50)
        
        # Add technical noise
        noise = np.random.normal(0, 0.1, base_expression.shape)
        expression_data = base_expression + noise
        expression_data = np.maximum(expression_data, 0)  # No negative expression
        
        all_data.append(expression_data)
        all_labels.extend([condition] * n_samples)
    
    data = np.vstack(all_data)
    labels = np.array(all_labels)
    
    # Log-transform (common preprocessing for gene expression)
    data = np.log1p(data)
    
    return data, labels, gene_names

# Generate data
data, labels, gene_names = generate_gene_expression_data(
    n_samples=200, n_genes=2000, n_informative=200
)

print(f"Generated gene expression data:")
print(f"  Shape: {data.shape} (samples x genes)")
print(f"  Conditions: 0 (n={np.sum(labels==0)}), 1 (n={np.sum(labels==1)}), 2 (n={np.sum(labels==2)})")
print(f"  Expression range: [{data.min():.2f}, {data.max():.2f}]")

## 3. Data Preprocessing and Visualization

Before building the autoencoder, let's:
1. Standardize the data (zero mean, unit variance)
2. Visualize the data distribution
3. Check correlations

In [ ]:
# Standardize data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Visualize expression distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Distribution before scaling
axes[0].hist(data.flatten(), bins=50, alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Log Expression')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Expression Distribution (Original)')
axes[0].grid(True, alpha=0.3)

# Distribution after scaling
axes[1].hist(data_scaled.flatten(), bins=50, alpha=0.7, edgecolor='black', color='orange')
axes[1].set_xlabel('Scaled Expression')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Expression Distribution (Scaled)')
axes[1].grid(True, alpha=0.3)

# Sample-wise mean expression
for condition in range(3):
    condition_data = data[labels == condition].mean(axis=1)
    axes[2].hist(condition_data, bins=20, alpha=0.5, 
                label=f'Condition {condition}', edgecolor='black')
axes[2].set_xlabel('Mean Expression per Sample')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Expression by Condition')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Create PyTorch Dataset and DataLoaders

In [ ]:
# Convert to PyTorch tensors
X_tensor = torch.FloatTensor(data_scaled)
y_tensor = torch.LongTensor(labels)

# Create dataset
dataset = TensorDataset(X_tensor, y_tensor)

# Split into train and test (80/20)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Dataset splits:")
print(f"  Train: {len(train_dataset)} samples")
print(f"  Test: {len(test_dataset)} samples")

## 5. Build the Autoencoder

### Autoencoder Architecture:

An autoencoder has two parts:

1. **Encoder**: Compresses input to a lower-dimensional representation (bottleneck/latent space)
   - Input: 2000 genes
   - Hidden layers: 1024 → 512 → 256
   - Latent space: 32 dimensions

2. **Decoder**: Reconstructs the original input from the latent representation
   - Latent: 32 dimensions
   - Hidden layers: 256 → 512 → 1024
   - Output: 2000 genes

The model learns by trying to reconstruct its input, forcing it to learn meaningful patterns!

In [ ]:
class GeneExpressionAutoencoder(nn.Module):
    """
    Autoencoder for gene expression dimensionality reduction.
    """
    def __init__(self, input_dim=2000, latent_dim=32):
        super(GeneExpressionAutoencoder, self).__init__()
        
        # Encoder: Compresses data to latent representation
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),  # Normalizes activations
            nn.Dropout(0.2),
            
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.2),
            
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.2),
            
            nn.Linear(256, latent_dim)  # Bottleneck layer
        )
        
        # Decoder: Reconstructs data from latent representation
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.2),
            
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.2),
            
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.2),
            
            nn.Linear(1024, input_dim)  # Reconstruct original dimensions
        )
    
    def forward(self, x):
        # Encode
        latent = self.encoder(x)
        # Decode
        reconstructed = self.decoder(latent)
        return reconstructed
    
    def encode(self, x):
        """Get latent representation only."""
        return self.encoder(x)

# Create model
input_dim = data_scaled.shape[1]
latent_dim = 32
model = GeneExpressionAutoencoder(input_dim=input_dim, latent_dim=latent_dim).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(model)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"\nCompression ratio: {input_dim}/{latent_dim} = {input_dim/latent_dim:.1f}x")

## 6. Training Setup

### Loss Function: Mean Squared Error (MSE)
Measures how well the reconstruction matches the original input. Lower MSE means better reconstruction.

### Why MSE for Autoencoders?
We want to minimize the difference between input and reconstructed output.

In [ ]:
# Loss function: MSE for reconstruction
criterion = nn.MSELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5, verbose=True
)

## 7. Training Loop

The autoencoder learns to reconstruct its input. The better it compresses and reconstructs, the better it has learned the underlying structure!

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    """
    Train autoencoder for one epoch.
    """
    model.train()
    total_loss = 0
    
    for batch_data, _ in tqdm(loader, desc="Training"):
        batch_data = batch_data.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass: try to reconstruct input
        reconstructed = model(batch_data)
        
        # Calculate reconstruction loss
        loss = criterion(reconstructed, batch_data)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)

def validate(model, loader, criterion, device):
    """
    Validate autoencoder.
    """
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch_data, _ in loader:
            batch_data = batch_data.to(device)
            reconstructed = model(batch_data)
            loss = criterion(reconstructed, batch_data)
            total_loss += loss.item()
    
    return total_loss / len(loader)

# Training
num_epochs = 50
train_losses = []
test_losses = []

print("Starting training...\n")

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    test_loss = validate(model, test_loader, criterion, device)
    
    scheduler.step(test_loss)
    
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Train Loss: {train_loss:.6f}")
        print(f"  Test Loss: {test_loss:.6f}\n")

print("Training completed!")

## 8. Visualize Training Progress

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Train Loss', marker='o', markersize=3)
plt.plot(test_losses, label='Test Loss', marker='s', markersize=3)
plt.xlabel('Epoch')
plt.ylabel('Reconstruction Loss (MSE)')
plt.title('Autoencoder Training Progress')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Final training loss: {train_losses[-1]:.6f}")
print(f"Final test loss: {test_losses[-1]:.6f}")

## 9. Extract Latent Representations

Now let's use the trained encoder to get 32-dimensional representations of all samples.

In [ ]:
def get_latent_representations(model, data_tensor, device):
    """
    Extract latent representations from trained autoencoder.
    """
    model.eval()
    with torch.no_grad():
        data_tensor = data_tensor.to(device)
        latent = model.encode(data_tensor)
    return latent.cpu().numpy()

# Get latent representations for all data
latent_representations = get_latent_representations(model, X_tensor, device)

print(f"Latent representations shape: {latent_representations.shape}")
print(f"Original data shape: {data_scaled.shape}")
print(f"Dimensionality reduction: {data_scaled.shape[1]} → {latent_representations.shape[1]}")

## 10. Visualize Latent Space with t-SNE

We'll use t-SNE to visualize the 32-dimensional latent space in 2D. If the autoencoder learned well, samples from the same condition should cluster together.

In [ ]:
# Apply t-SNE for visualization
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
latent_tsne = tsne.fit_transform(latent_representations)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Autoencoder latent space
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
condition_names = ['Healthy', 'Disease 1', 'Disease 2']

for i, (color, name) in enumerate(zip(colors, condition_names)):
    mask = labels == i
    axes[0].scatter(latent_tsne[mask, 0], latent_tsne[mask, 1], 
                   c=color, label=name, alpha=0.6, s=50, edgecolors='black', linewidth=0.5)

axes[0].set_xlabel('t-SNE Dimension 1')
axes[0].set_ylabel('t-SNE Dimension 2')
axes[0].set_title('Autoencoder Latent Space (32D → 2D via t-SNE)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Compare with PCA on original data
pca = PCA(n_components=2)
pca_result = pca.fit_transform(data_scaled)

for i, (color, name) in enumerate(zip(colors, condition_names)):
    mask = labels == i
    axes[1].scatter(pca_result[mask, 0], pca_result[mask, 1], 
                   c=color, label=name, alpha=0.6, s=50, edgecolors='black', linewidth=0.5)

axes[1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
axes[1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
axes[1].set_title('PCA on Original Data (2000D → 2D)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservations:")
print("- Autoencoder learns non-linear relationships")
print("- PCA only captures linear relationships")
print("- Both show separation of conditions, but autoencoder may capture more complex patterns")

## 11. Clustering in Latent Space

Let's use K-means clustering on the latent representations to see if we can recover the three conditions.

In [ ]:
# K-means clustering
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(latent_representations)

# Calculate clustering accuracy (best match between clusters and true labels)
from scipy.optimize import linear_sum_assignment

def clustering_accuracy(true_labels, cluster_labels):
    """
    Calculate clustering accuracy by finding best label assignment.
    """
    # Create confusion matrix
    n_clusters = len(np.unique(cluster_labels))
    n_classes = len(np.unique(true_labels))
    confusion = np.zeros((n_classes, n_clusters), dtype=int)
    
    for i in range(len(true_labels)):
        confusion[true_labels[i], cluster_labels[i]] += 1
    
    # Find best assignment using Hungarian algorithm
    row_ind, col_ind = linear_sum_assignment(-confusion)
    
    accuracy = confusion[row_ind, col_ind].sum() / len(true_labels)
    return accuracy, dict(zip(col_ind, row_ind))

accuracy, mapping = clustering_accuracy(labels, cluster_labels)

print(f"Clustering accuracy: {accuracy:.2%}")
print(f"\nCluster to condition mapping: {mapping}")

# Visualize clusters
plt.figure(figsize=(10, 6))
for i in range(3):
    mask = cluster_labels == i
    plt.scatter(latent_tsne[mask, 0], latent_tsne[mask, 1], 
               label=f'Cluster {i}', alpha=0.6, s=50, edgecolors='black', linewidth=0.5)

plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.title(f'K-means Clustering in Latent Space (Accuracy: {accuracy:.1%})')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 12. Visualize Reconstruction Quality

Let's check how well the autoencoder reconstructs the original gene expression profiles.

In [ ]:
# Get reconstructions
model.eval()
with torch.no_grad():
    X_tensor_gpu = X_tensor.to(device)
    reconstructed = model(X_tensor_gpu).cpu().numpy()

# Plot original vs reconstructed for a few samples
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.ravel()

sample_indices = np.random.choice(len(data_scaled), 6, replace=False)

for idx, sample_idx in enumerate(sample_indices):
    original = data_scaled[sample_idx]
    recon = reconstructed[sample_idx]
    condition = labels[sample_idx]
    
    # Plot first 100 genes
    axes[idx].plot(original[:100], label='Original', alpha=0.7, linewidth=1)
    axes[idx].plot(recon[:100], label='Reconstructed', alpha=0.7, linewidth=1)
    axes[idx].set_xlabel('Gene Index')
    axes[idx].set_ylabel('Scaled Expression')
    axes[idx].set_title(f'Sample {sample_idx} (Condition {condition})')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate reconstruction error
mse = np.mean((data_scaled - reconstructed) ** 2)
print(f"\nMean reconstruction error (MSE): {mse:.6f}")

## 13. Identify Important Features in Latent Space

We can analyze the decoder weights to understand which genes are most important for each latent dimension.

In [ ]:
# Get decoder weights (last layer)
decoder_weights = model.decoder[-1].weight.data.cpu().numpy()  # Shape: (n_genes, latent_dim)

# For each latent dimension, find top genes
n_top_genes = 10
print("Top genes associated with each latent dimension:\n")

for dim in range(min(5, latent_dim)):  # Show first 5 dimensions
    weights = decoder_weights[:, dim]
    top_indices = np.argsort(np.abs(weights))[-n_top_genes:]
    
    print(f"Latent Dimension {dim}:")
    for idx in reversed(top_indices):
        print(f"  {gene_names[idx]}: {weights[idx]:.4f}")
    print()

# Visualize latent dimension importance
latent_variance = np.var(latent_representations, axis=0)
sorted_indices = np.argsort(latent_variance)[::-1]

plt.figure(figsize=(12, 5))
plt.bar(range(latent_dim), latent_variance[sorted_indices])
plt.xlabel('Latent Dimension (sorted by variance)')
plt.ylabel('Variance')
plt.title('Variance Explained by Each Latent Dimension')
plt.grid(True, alpha=0.3, axis='y')
plt.show()

print(f"\nTop 10 latent dimensions explain {100 * latent_variance[sorted_indices[:10]].sum() / latent_variance.sum():.1f}% of variance")

## Summary and Key Takeaways

In this notebook, we:

1. ✅ **Built an autoencoder** for unsupervised dimensionality reduction
2. ✅ **Compressed** 2000-dimensional gene expression to 32 dimensions
3. ✅ **Visualized** the learned latent space using t-SNE
4. ✅ **Compared** with traditional PCA
5. ✅ **Performed clustering** in the latent space
6. ✅ **Analyzed** reconstruction quality

### Why Autoencoders for Gene Expression?

- **Non-linear dimensionality reduction**: Captures complex patterns PCA misses
- **Unsupervised learning**: No labels needed
- **Denoising**: Can be trained to remove technical noise
- **Feature learning**: Discovers biologically meaningful representations
- **Scalability**: Handles high-dimensional data efficiently

### Advanced Techniques:

- **Variational Autoencoders (VAE)**: Add probabilistic structure
- **Denoising Autoencoders**: Train with corrupted input
- **Sparse Autoencoders**: Encourage sparse activations
- **Adversarial Autoencoders**: Use GANs for better latent space

### Real-World Applications:

- **Cancer subtype discovery**: Find new cancer subtypes from expression
- **Drug response prediction**: Predict treatment outcomes
- **Single-cell RNA-seq**: Analyze individual cell transcriptomes
- **Batch effect correction**: Remove technical variation
- **Data imputation**: Fill in missing values
- **Biomarker discovery**: Identify diagnostic markers